In [1]:
"""
GOAL: GENERATE THE MailData Spreadsheet for time efficiency
Rough Draft:
Generate the following fields, 
Total Mail Pieces:
Calls:
Response Rate:
Locks:
Fundings:
Call to Lock:
Lock to Fund:

There are 3 marketing ads: Camber, Amsive, Redstone
"""

'\nGOAL: GENERATE THE MailData Spreadsheet for time efficiency\nRough Draft:\nGenerate the following fields, \nTotal Mail Pieces:\nCalls:\nResponse Rate:\nLocks:\nFundings:\nCall to Lock:\nLock to Fund:\n\nThere are 3 marketing ads: Camber, Amsive, Redstone\n'

In [1]:
# load environment variable to get .env file containing email and password
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env")

# import all necessary packages
import os
import time
import sys
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

#add src folder to path
sys.path.append(os.path.abspath("../src"))

from scraping.driver_setup import get_driver
from scraping.login import login
from scraping.leads_scraper import scrape_leads
from scraping.pipelinemetrics_scraper import scrape_pipeline_metrics
from scraping.loanstatus_scraper import scrape_loanstatus
from utils.file_ops import rename_latest_file

from selenium.webdriver.support.ui import WebDriverWait

# Set download directory
download_dir = os.path.abspath("../../data/raw")
# Launch browser
driver = get_driver(download_dir)
wait = WebDriverWait(driver, 15)

# Run scraping tasks
login(driver, wait)


#Scrape Leads_CRM
scrape_leads(driver, wait, download_dir)
rename_latest_file(download_dir, "Leads_CRM.csv")

#Scrape Loanstatus
scrape_loanstatus(driver, wait, download_dir)
rename_latest_file(download_dir, "Loanstatus.csv")
#Scrape PipelineMetrics_LOS
scrape_pipeline_metrics(driver, wait, download_dir)
rename_latest_file(download_dir, "PipelineMetrics_LOS.csv")


🔗 Navigated to login page
➡️ Submitted username
🔐 Clicked login button
✅ Login attempt complete
📂 'My Lists' clicked
✅ Leads.csv downloaded and renamed.
✅ Loanstatus.csv downloaded and renamed.
1
✅ 'Reporting' menu clicked
✅ 'Pipeline Metrics' clicked
✅ Outer 'This Week' filter clicked
✅ Inner 'This Week' dropdown clicked
✅ 'Custom' date range selected
📅 'From' date set to 03/28/2024
[0]  → 
[1] Branch → 29: branch
[2] Loan Status → 30: loanStatus
[3] Loan Purpose → 31: loanPurpose
[4] Loan Type → 32: loanType
[5] Loan Officer → 33: primaryRoleUser
[6] Lead Source → 34: leadSource
[7] State → 35: state
[8] Sub Status → 36: subStatus
[9] Referral Source → 37: referralSource
[10] Inside Referral Source → 38: insideReferralSource
[11] Referred To → 39: referredTo
[12] Lead Campaign Name → 40: leadCampaignName
[13] Secondary Referral Source → 41: secondaryReferralSource
[14] Lead Ref ID → 42: leadRefId
✅ 'Lead Ref ID' selected by visible text
📂 First accordion clicked
🧩 Columns dropdown op

In [65]:
# Re-fetch the dropdown inside Pipeline Metrics section
dropdown = wait.until(EC.presence_of_element_located(
    (By.XPATH, "//h4[.//span[text()='Pipeline Metrics']]//select"))
)
select = Select(dropdown)

# Log all options to confirm visibility
for i, opt in enumerate(select.options):
    print(f"[{i}] {opt.text.strip()} → {opt.get_attribute('value')}")

# Select by visible text (robust against value changes)
select.select_by_visible_text("Lead Ref ID")
print("✅ 'Lead Ref ID' selected by visible text")
time.sleep(1)

[0]  → 
[1] Branch → 29: branch
[2] Loan Status → 30: loanStatus
[3] Loan Purpose → 31: loanPurpose
[4] Loan Type → 32: loanType
[5] Loan Officer → 33: primaryRoleUser
[6] Lead Source → 34: leadSource
[7] State → 35: state
[8] Sub Status → 36: subStatus
[9] Referral Source → 37: referralSource
[10] Inside Referral Source → 38: insideReferralSource
[11] Referred To → 39: referredTo
[12] Lead Campaign Name → 40: leadCampaignName
[13] Secondary Referral Source → 41: secondaryReferralSource
[14] Lead Ref ID → 42: leadRefId
✅ 'Lead Ref ID' selected by visible text


In [ ]:


    # Step 5: Trigger CSV export
    export_button = wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//button[contains(text(), 'Export To CSV')]"))
    )
    export_button.click()
    print("📤 Export triggered")
    time.sleep(2)

    # Step 6: Wait for download to finish
    start_time = time.time()
    while any(f.endswith(".crdownload") for f in os.listdir(download_dir)):
        if time.time() - start_time > 30:
            raise TimeoutError("Download did not complete within expected time.")
        time.sleep(0.5)

    # Step 7: Rename the most recent download
    files = sorted(os.listdir(download_dir), key=lambda x: os.path.getctime(os.path.join(download_dir, x)), reverse=True)
    latest_file = files[0]
    os.rename(os.path.join(download_dir, latest_file), os.path.join(download_dir, "PipelineMetrics_LOS.csv"))
    print("✅ PipelineMetrics_LOS.csv downloaded and renamed.")


driver.quit()

In [ ]:

#Scrape PipelineMetrics_LOS
scrape_pipeline_metrics(driver, wait, download_dir)
rename_latest_file(download_dir, "PipelineMetrics_LOS.csv")

#end session
driver.quit()

#import sys
sys.path.append("..")

from sync_to_google_sheets import upload_csv_to_sheet

# Upload Leads_CRM.csv to 'CRM' tab
upload_csv_to_sheet("../../data/raw/Leads_CRM.csv", "CRM")
# Upload PipelineMetrics_LOS.csv to 'LOS' tab
upload_csv_to_sheet("../../data/raw/PipelineMetrics_LOS.csv", "LOS")
# Upload PipelineMetrics_LOS.csv to 'LoanStatus' tab
upload_csv_to_sheet("../../data/raw/LoanStatus.csv", "LoanStatus")



In [ ]:
#import sys
sys.path.append("..")

from sync_to_google_sheets import upload_csv_to_sheet

# Upload Leads_CRM.csv to 'CRM' tab
upload_csv_to_sheet("../../data/raw/Leads_CRM.csv", "CRM")
# Upload PipelineMetrics_LOS.csv to 'LOS' tab
upload_csv_to_sheet("../../data/raw/PipelineMetrics_LOS.csv", "LOS")
# Upload PipelineMetrics_LOS.csv to 'LoanStatus' tab
upload_csv_to_sheet("../../data/raw/LoanStatus.csv", "LoanStatus")


✅ Uploaded ../../data/raw/Leads_CRM.csv to tab 'CRM'
✅ Uploaded ../../data/raw/PipelineMetrics_LOS.csv to tab 'LOS'
✅ Uploaded ../../data/raw/LoanStatus.csv to tab 'LoanStatus'
